<a href="https://colab.research.google.com/github/dannydoan43/UFCPredictor/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
fighterdata = pd.read_csv('fighters.csv')
boutdata = pd.read_csv('bouts.csv')


,date_of_birth,fighter_name,fighter_record,height,reach,sapm,slpm,stance,strike_acc,strike_def,sub_avg,td_acc,td_avg,td_def,weight
0,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,185.0
1,Jun 16 1977,Rich Attonito,Record: 10-5-0,"5' 10""",72.0,2.13,2.28,Orthodox,47,59.0,0.0,37,0.74,75,170.0
2,Feb 07 1989,Gleidson Cutis,Record: 7-4-0,"5' 9""",NaN,8.28,2.99,Orthodox,52,59.0,0.0,0,0.00,0,155.0
3,Feb 05 1976,Antoni Hardonk,Record: 8-6-0,"6' 4""",78.0,3.07,3.40,Orthodox,61,46.0,0.9,0,0.00,40,245.0
4,Jul 13 1978,Tom Aaron,Record: 5-3-0,--,NaN,0.00,0.00,NaN,0,0.0,0.0,0,0.00,0,155.0


In [ ]:
fighterdata = fighterdata[fighterdata['date_of_birth'] != '--']
fighterdata = fighterdata[fighterdata['height'] != '--']
fighterdata = fighterdata.drop('weight',1)
fighterdata[fighterdata.duplicated(subset="fighter_name", keep=False)]

#2990 x15 before 2936 x14 after
display(fighterdata)
#might want to include the winmethodtype
boutdata = boutdata.drop(columns=['event_name','event_date', 'win_method_finish','time','win_method_type'])
display(boutdata)
#6376x18 before 6376x16 after


,date_of_birth,fighter_name,fighter_record,height,reach,sapm,slpm,stance,strike_acc,strike_def,sub_avg,td_acc,td_avg,td_def
0,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65
1,Jun 16 1977,Rich Attonito,Record: 10-5-0,"5' 10""",72.0,2.13,2.28,Orthodox,47,59.0,0.0,37,0.74,75
2,Feb 07 1989,Gleidson Cutis,Record: 7-4-0,"5' 9""",NaN,8.28,2.99,Orthodox,52,59.0,0.0,0,0.00,0
3,Feb 05 1976,Antoni Hardonk,Record: 8-6-0,"6' 4""",78.0,3.07,3.40,Orthodox,61,46.0,0.9,0,0.00,40
5,Dec 14 1993,Ion Cutelaba,Record: 16-6-1 (1 NC),"6' 1""",75.0,3.43,4.91,Southpaw,42,47.0,0.0,60,4.39,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3728,Nov 07 1990,Carlos Ulberg,Record: 5-1-0,"6' 4""",77.0,8.60,15.07,Orthodox,62,47.0,0.0,0,0.00,100
3729,Aug 04 1985,Christian Uflacker,Record: 5-0-0,"5' 11""",NaN,0.00,0.00,NaN,0,0.0,0.0,0,0.00,0
3730,Aug 07 1991,Tagir Ulanbekov,Record: 14-1-0,"5' 7""",70.0,2.40,2.13,Southpaw,39,52.0,0.5,56,4.50,55
3731,Apr 17 1976,Ryuki Ueyama,Record: 12-18-5 (1 NC),"5' 11""",NaN,1.40,0.13,Southpaw,21,46.0,1.5,100,0.50,0


,fighter1,fighter2,pass_stat_f1,pass_stat_f2,round_,str_stat_f1,str_stat_f2,sub_stat_f1,sub_stat_f2,td_stat_f1,td_stat_f2,weight_class,winner
0,Curtis Blaydes,Junior Dos Santos,0,0,2,0,0,0,0,28,7,Heavyweight,Curtis Blaydes
1,Michael Chiesa,Rafael Dos Anjos,0,0,3,0,0,6,0,9,14,Welterweight,Michael Chiesa
2,Alex Perez,Jordan Espinosa,1,0,1,0,0,1,0,2,2,Flyweight,Alex Perez
3,Angela Hill,Hannah Cifers,0,0,2,0,0,1,0,70,46,Women's Strawweight,Angela Hill
4,Jamahal Hill,Darko Stosic,0,0,3,0,0,0,6,101,31,Light Heavyweight,Jamahal Hill
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6371,Andrea Lee,Cynthia Calvillo,0,0,2,0,0,0,1,84,47,Women's Flyweight,Andrea Lee
6372,Sean Woodson,Collin Anglin,0,0,1,1,0,0,0,50,8,Featherweight,Sean Woodson
6373,Cortney Casey,Liana Jojua,0,0,3,0,0,0,2,89,34,Women's Flyweight,Cortney Casey
6374,Rafael Alves,Marc Diakiese,1,0,1,0,0,0,0,6,3,Lightweight,Rafael Alves


In [ ]:
fighterdata.iloc[513, 1] = "Michael McDonald 205"
fighterdata.iloc[845, 1] = "Dong Hyun Kim 155"
fighterdata.iloc[954, 1] = "Tony Johnson 185"
fighterdata.iloc[1674, 1] = "Mike Davis 145"

In [ ]:
fighterdata.set_index("fighter_name", inplace=True)

In [ ]:
fighers_clean = fighterdata.loc[~((fighterdata["slpm"] == 0) &
                               (fighterdata["strike_acc"] == "0%") & 
                               (fighterdata["sapm"] == 0) &
                               (fighterdata["strike_def"] == "0%") &
                               (fighterdata["td_avg"] == 0) &
                               (fighterdata["td_acc"] == "0%") &
                               (fighterdata["td_def"] == "0%") &
                               (fighterdata["sub_avg"] == 0))].copy()  

In [ ]:
print("{0} fighers in total, after clean up: {1} fighers".format(len(fighterdata), len(fighers_clean)))

2936 fighers in total, after clean up: 2936 fighers


In [ ]:
#split fighter_record into win loss nc

In [ ]:
# add winning percentages
fighers_clean["win%"] = 100 * (fighers_clean["win"] / (fighers_clean["win"] +
                                                 fighers_clean["lose"] +
                                                 fighers_clean["draw"] +
                                                 fighers_clean["nc"]))
# change datatypes
percentages = ["strike_acc", "strike_def", "td_acc", "td_def"]
statistics = ["slpm", "strike_acc", "sapm", "strike_def", "td_avg", "td_acc", "td_def", "sub_avg", "win%"]

fighers_clean.loc[:, percentages] = fighers_clean.loc[:, percentages].applymap(
    lambda x: x.replace("%", ""))

fighers_clean.loc[:, statistics] = fighers_clean.loc[:, statistics].astype(np.float32)

KeyError: ignored

In [ ]:
datatemp = pd.merge(fighterdata,boutdata, left_on="fighter_name", right_on="fighter2")
data = pd.merge(fighterdata,datatemp, left_on="fighter_name", right_on="fighter1")
data.to_csv('test.csv')

In [ ]:
print(data.head(15))
print(data.shape)

display(data)

   date_of_birth_x  fighter_name_x  ...  weight_class          winner
0      Oct 12 1987  Omari Akhmedov  ...  Welterweight  Omari Akhmedov
1      Oct 12 1987  Omari Akhmedov  ...  Middleweight  Omari Akhmedov
2      Oct 12 1987  Omari Akhmedov  ...  Welterweight  Omari Akhmedov
3      Oct 12 1987  Omari Akhmedov  ...  Welterweight  Omari Akhmedov
4      Oct 12 1987  Omari Akhmedov  ...  Middleweight  Omari Akhmedov
5      Oct 12 1987  Omari Akhmedov  ...  Middleweight  Omari Akhmedov
6      Oct 12 1987  Omari Akhmedov  ...  Middleweight  Omari Akhmedov
7      Oct 12 1987  Omari Akhmedov  ...  Middleweight  Omari Akhmedov
8      Oct 12 1987  Omari Akhmedov  ...  Welterweight  Omari Akhmedov
9      Jun 16 1977   Rich Attonito  ...  Welterweight   Rich Attonito
10     Jun 16 1977   Rich Attonito  ...  Middleweight   Rich Attonito
11     Jun 16 1977   Rich Attonito  ...  Middleweight   Rich Attonito
12     Feb 05 1976  Antoni Hardonk  ...   Heavyweight  Antoni Hardonk
13     Feb 05 1976  

,date_of_birth_x,fighter_name_x,fighter_record_x,height_x,reach_x,sapm_x,slpm_x,stance_x,strike_acc_x,strike_def_x,sub_avg_x,td_acc_x,td_avg_x,td_def_x,date_of_birth_y,fighter_name_y,fighter_record_y,height_y,reach_y,sapm_y,slpm_y,stance_y,strike_acc_y,strike_def_y,sub_avg_y,td_acc_y,td_avg_y,td_def_y,fighter1,fighter2,pass_stat_f1,pass_stat_f2,round_,str_stat_f1,str_stat_f2,sub_stat_f1,sub_stat_f2,td_stat_f1,td_stat_f2,weight_class,winner
0,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Aug 11 1985,Abdul Razak Alhassan,Record: 11-4-0,"5' 10""",73.0,4.23,3.71,Orthodox,48,53.0,0.0,28,0.53,55,Omari Akhmedov,Abdul Razak Alhassan,0,0,3,0,0,6,0,52,43,Welterweight,Omari Akhmedov
1,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Dec 14 1987,Thiago Perpetuo,Record: 9-4-1,"5' 10""",NaN,3.51,5.39,Orthodox,69,51.0,0.0,50,2.82,66,Omari Akhmedov,Thiago Perpetuo,1,0,1,1,1,1,1,14,18,Middleweight,Omari Akhmedov
2,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Mar 18 1980,Kyle Noke,Record: 22-10-1,"6' 1""",76.0,2.09,3.23,Southpaw,50,64.0,0.8,54,1.53,66,Omari Akhmedov,Kyle Noke,0,0,3,0,0,3,0,23,16,Welterweight,Omari Akhmedov
3,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Dec 05 1983,Mats Nilsson,Record: 11-4-1,"6' 1""",NaN,5.62,2.99,Orthodox,39,63.0,2.3,83,3.87,44,Omari Akhmedov,Mats Nilsson,0,3,3,0,0,5,5,64,33,Welterweight,Omari Akhmedov
4,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Sep 26 1991,Tom Breese,Record: 12-3-0,"6' 3""",73.0,2.81,3.34,Southpaw,50,60.0,1.1,0,0.00,70,Omari Akhmedov,Tom Breese,1,3,2,0,0,2,0,25,1,Middleweight,Omari Akhmedov
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6194,Dec 13 1993,Justin Tafa,Record: 4-3-0,"6' 0""",74.0,6.12,5.15,Southpaw,55,49.0,0.0,0,0.00,100,Jan 16 1992,Juan Adams,Record: 5-3-0,"6' 5""",80.0,4.06,7.09,Orthodox,55,34.0,0.0,66,0.91,57,Justin Tafa,Juan Adams,0,0,1,1,0,0,0,10,7,Heavyweight,Justin Tafa
6195,Sep 11 1982,Gasan Umalatov,Record: 17-4-2,"5' 11""",71.0,4.10,2.23,Orthodox,39,59.0,0.0,17,0.75,90,Jan 25 1981,Paulo Thiago,Record: 18-9-0,"5' 11""",74.0,1.99,1.71,Orthodox,39,63.0,1.2,31,1.42,63,Gasan Umalatov,Paulo Thiago,0,0,3,0,0,1,1,24,42,Welterweight,Gasan Umalatov
6196,Aug 07 1991,Tagir Ulanbekov,Record: 14-1-0,"5' 7""",70.0,2.40,2.13,Southpaw,39,52.0,0.5,56,4.50,55,Sep 11 1991,Allan Nascimento,Record: 17-6-0,"5' 8""",69.0,3.30,4.33,Orthodox,68,46.0,1.0,9,0.50,16,Tagir Ulanbekov,Allan Nascimento,1,2,3,0,0,4,0,18,25,Flyweight,Tagir Ulanbekov
6197,Aug 07 1991,Tagir Ulanbekov,Record: 14-1-0,"5' 7""",70.0,2.40,2.13,Southpaw,39,52.0,0.5,56,4.50,55,Jul 13 1989,Bruno Silva,Record: 21-6-0,"6' 0""",74.0,1.96,5.13,Orthodox,59,37.0,0.0,0,0.00,68,Tagir Ulanbekov,Bruno Silva,0,0,3,0,0,5,4,46,47,Flyweight,Tagir Ulanbekov


In [ ]:
data.head()

,date_of_birth_x,fighter_name_x,fighter_record_x,height_x,reach_x,sapm_x,slpm_x,stance_x,strike_acc_x,strike_def_x,sub_avg_x,td_acc_x,td_avg_x,td_def_x,date_of_birth_y,fighter_name_y,fighter_record_y,height_y,reach_y,sapm_y,slpm_y,stance_y,strike_acc_y,strike_def_y,sub_avg_y,td_acc_y,td_avg_y,td_def_y,fighter1,fighter2,pass_stat_f1,pass_stat_f2,round_,str_stat_f1,str_stat_f2,sub_stat_f1,sub_stat_f2,td_stat_f1,td_stat_f2,weight_class,winner
0,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Aug 11 1985,Abdul Razak Alhassan,Record: 11-4-0,"5' 10""",73.0,4.23,3.71,Orthodox,48,53.0,0.0,28,0.53,55,Omari Akhmedov,Abdul Razak Alhassan,0,0,3,0,0,6,0,52,43,Welterweight,Omari Akhmedov
1,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Dec 14 1987,Thiago Perpetuo,Record: 9-4-1,"5' 10""",NaN,3.51,5.39,Orthodox,69,51.0,0.0,50,2.82,66,Omari Akhmedov,Thiago Perpetuo,1,0,1,1,1,1,1,14,18,Middleweight,Omari Akhmedov
2,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Mar 18 1980,Kyle Noke,Record: 22-10-1,"6' 1""",76.0,2.09,3.23,Southpaw,50,64.0,0.8,54,1.53,66,Omari Akhmedov,Kyle Noke,0,0,3,0,0,3,0,23,16,Welterweight,Omari Akhmedov
3,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Dec 05 1983,Mats Nilsson,Record: 11-4-1,"6' 1""",NaN,5.62,2.99,Orthodox,39,63.0,2.3,83,3.87,44,Omari Akhmedov,Mats Nilsson,0,3,3,0,0,5,5,64,33,Welterweight,Omari Akhmedov
4,Oct 12 1987,Omari Akhmedov,Record: 21-6-1,"6' 0""",73.0,2.84,2.97,Orthodox,34,55.0,0.2,45,2.58,65,Sep 26 1991,Tom Breese,Record: 12-3-0,"6' 3""",73.0,2.81,3.34,Southpaw,50,60.0,1.1,0,0.00,70,Omari Akhmedov,Tom Breese,1,3,2,0,0,2,0,25,1,Middleweight,Omari Akhmedov


In [ ]:
cats = ["fighter1", "fighter2", "weight_class", "method", "end_round", "fight_year",
        "height_fighter1", "reach_fighter1", "stance_fighter1", "born_year_fighter1",
        "height_fighter2", "reach_fighter2", "stance_fighter2", "born_year_fighter2"]

for cat in cats:
    data[cat].fillna("unk", inplace=True)
    data[cat] = data[cat].astype("category")

KeyError: ignored